# Transfromer test

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 63.6 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
! pip install git+https://github.com/huggingface/huggingface_hub.git@main
! sudo apt -qq install git-lfs
! git config --global credential.helper store

  Cloning https://github.com/huggingface/huggingface_hub.git (to revision main) to /tmp/pip-req-build-o05wkd9d
  Running command git clone -q https://github.com/huggingface/huggingface_hub.git /tmp/pip-req-build-o05wkd9d
  Created wheel for huggingface-hub: filename=huggingface_hub-0.4.0.dev0-py3-none-any.whl size=67068 sha256=b1d494192e476665b21f258158334f626aa8fdf224fd4667a90cf12ba6631436
  Stored in directory: /tmp/pip-ephem-wheel-cache-ltmc4ibj/wheels/8e/b6/fe/0f5fd2d49f7fd04a6f2437fb96643a08ac5ec70e7220ad42e0
Successfully built huggingface-hub
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.4.0
    Uninstalling huggingface-hub-0.4.0:
      Successfully uninstalled huggingface-hub-0.4.0
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
debconf: unable to initialize fro

In [5]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


## Liberias

In [16]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List, Optional
from tensorflow.keras import models
from huggingface_hub import (from_pretrained_keras,
                             push_to_hub_keras)
from transformers import (DistilBertConfig,
                          DistilBertTokenizer,
                          TFDistilBertForSequenceClassification)

In [7]:
os.chdir("/content/drive/MyDrive/GovContracts")

## Encoders

In [8]:
encoder = pickle.load(open('sell2sam/encoder.pkl', 'rb'))

## Model

### Transfer learning

### Tokenizer

In [9]:
tokenizer = DistilBertTokenizer(vocab_file = 'sell2sam/vocab.txt',
                                tokenizer_file = 'sell2sam/tokenizer.json',
                                tokenizer_config = 'sell2sam/tokenizer_config.json',
                                special_tokens_map = 'sell2sam/special_tokens_map.json',
                                do_lower_case = True,
                                add_special_tokens = True,
                                max_length = 512
                                )

In [10]:
def tokenize(sentences: List[str], tokenizer):

  inputs_id, inputs_masks = list(), list()

  for sentence in tqdm(sentences):

    inputs = tokenizer.encode_plus(sentence, add_special_tokens = True,
                                   max_length = 512, return_token_type_ids = True,
                                   return_attention_mask = True,
                                   pad_to_max_length = True)
    
    inputs_id.append(inputs["input_ids"])

    inputs_masks.append(inputs['attention_mask'])

  return np.asarray(inputs_id, dtype='int32'), \
         np.asarray(inputs_masks, dtype='int32') 

### Transformer

In [11]:
config = DistilBertConfig(num_labels = 1010)

config.output_hidden_states = False

transformer_model = TFDistilBertForSequenceClassification(config = config)

In [34]:
def build_model(file: Optional[str]= "model.json"):

  loaded_model = models.load_model("sell2sam/sell2sam.h5",
                                   custom_objects = {"TFDistilBertForSequenceClassification":
                                                     transformer_model})
  
  loaded_model.load_weights("sell2sam/sell2sam_weights.h5")

  loaded_model.build([(None, 512),
                      None, 512])

  return loaded_model

model = build_model()

### Save as API

In [35]:
push_to_hub_keras(model, '/Unlikeghost/sell2sam', private = True)

INFO:tensorflow:Assets written to: /Unlikeghost/sell2sam/assets


INFO:tensorflow:Assets written to: /Unlikeghost/sell2sam/assets
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file variables/variables.data-00000-of-00001:   0%|          | 3.39k/787M [00:00<?, ?B/s]

Upload file saved_model.pb:   0%|          | 3.39k/5.06M [00:00<?, ?B/s]

Upload file keras_metadata.pb:   1%|1         | 3.39k/270k [00:00<?, ?B/s]

To https://huggingface.co/Unlikeghost/sell2sam
   9331b37..e8e357a  main -> main

   9331b37..e8e357a  main -> main



'https://huggingface.co/Unlikeghost/sell2sam/commit/e8e357a5e8946acce2e69112b15f57a58c5bd820'

## Test

In [37]:
model2 = from_pretrained_keras('/Unlikeghost/sell2sam',
                               config = config,)

config.json not found in HuggingFace Hub


In [38]:
keywords = "software"

inputs_id_test, inputs_masks_inputs_id_test = tokenize([keywords],
                                                       tokenizer)

  0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1/1 [00:00<00:00, 87.00it/s]


In [40]:
preds = model2.predict([inputs_id_test, inputs_masks_inputs_id_test])

y_preds = np.argsort(preds, axis=1)[:,-5:]

y_preds = y_preds[0]

print(f"Top 5 results: { ', '.join(encoder.inverse_transform(y_preds))}")

Top 5 results: 924110, 518210, 423430, 541511, 513210


In [ ]:
preds = model.predict([inputs_id_test, inputs_masks_inputs_id_test])

y_preds = np.argsort(preds, axis=1)[:,-5:]

y_preds = y_preds[0]

print(f"Top 5 results: { ', '.join(encoder.inverse_transform(y_preds))}")

Top 5 results: 924110, 518210, 423430, 541511, 513210
